In [ ]:
# 下载历史数据（BTC/USDT，1h）
!freqtrade download-data \
    --config ../config.json \
    --userdir ../ \
    --pairs BTC/USDT \
    --timeframes 1h

In [1]:
# 读取已下载的数据到 DataFrame
from pathlib import Path

import pandas as pd
from freqtrade.data.history import load_pair_history

df = load_pair_history(datadir=Path("../data/okx"), timeframe="1h", pair="BTC/USDT")
df.tail()

,date,open,high,low,close,volume
714,2026-01-05 19:00:00+00:00,94451.1,94499.0,94133.6,94302.8,265.032684
715,2026-01-05 20:00:00+00:00,94302.8,94793.7,94034.2,94174.5,369.738545
716,2026-01-05 21:00:00+00:00,94175.4,94231.0,93801.8,94092.9,123.160190
717,2026-01-05 22:00:00+00:00,94092.9,94399.0,94008.0,94171.0,92.922083
718,2026-01-05 23:00:00+00:00,94171.1,94195.5,93858.4,93870.9,139.429309


In [ ]:
# 快速绘制K线图（默认样式）
from freqtrade.plot.plotting import generate_candlestick_graph

pair = "BTC/USDT"  # 绘图
graph = generate_candlestick_graph(data=df, pair=pair)
graph.show()


In [ ]:
# 计算 EMA/RSI 并绘图（最简版）
import numpy as np
import talib
from freqtrade.plot.plotting import generate_candlestick_graph

pair = "BTC/USDT"

close = df["close"].to_numpy(dtype=np.float64)

df["ema20"] = talib.EMA(close, timeperiod=20)
df["ema144"] = talib.EMA(close, timeperiod=144)
df["ema169"] = talib.EMA(close, timeperiod=169)
df["ema576"] = talib.EMA(close, timeperiod=576)
df["ema676"] = talib.EMA(close, timeperiod=676)
df["rsi"] = talib.RSI(close, timeperiod=14)
graph = generate_candlestick_graph(
    data=df,
    pair=pair,
    indicators1=["ema20", "ema144", "ema169", "ema576", "ema676"],
    indicators2=["rsi"],
)
graph.show()

In [ ]:
# 仅调整颜色（TradingView 风格）
import numpy as np

tv_green = "#26A69A"
tv_red = "#EF5350"

ema20_color = "#FBC02D"
ema_red = "#E53935"
ema_blue = "#1E88E5"
rsi_color = "#7E57C2"

graph.update_traces(
    increasing_line_color=tv_green,
    decreasing_line_color=tv_red,
    increasing_fillcolor="rgba(38, 166, 154, 0.25)",
    decreasing_fillcolor="rgba(239, 83, 80, 0.25)",
    selector={"type": "candlestick"},
)

graph.update_traces(line={"color": ema20_color}, selector={"name": "ema20"})
graph.update_traces(line={"color": ema_red}, selector={"name": "ema144"})
graph.update_traces(line={"color": ema_red}, selector={"name": "ema169"})
graph.update_traces(line={"color": ema_blue}, selector={"name": "ema576"})
graph.update_traces(line={"color": ema_blue}, selector={"name": "ema676"})

graph.update_traces(line={"color": rsi_color}, selector={"name": "rsi"})

vol_up = "rgba(38, 166, 154, 0.35)"
vol_down = "rgba(239, 83, 80, 0.35)"
volume_colors = np.where(df["close"] >= df["open"], vol_up, vol_down)
graph.update_traces(
    marker_color=volume_colors,
    marker_line_color=volume_colors,
    selector={"type": "bar", "name": "Volume"},
)

graph.show()

In [ ]:
# 买卖点绘图
from freqtrade.configuration import Configuration
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plot.plotting import generate_candlestick_graph
from freqtrade.resolvers import StrategyResolver

pair = "BTC/USDT"

config = Configuration.from_files(["D:\\Code\\python\\freqtrade_demo\\config.json"])

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

df = strategy.analyze_ticker(df, {"pair": pair})

# 绘图
graph = generate_candlestick_graph(
    data=df, pair=pair, indicators1=["ma_short", "ma_long"]
)
graph.show()

In [2]:
#回测
!freqtrade backtesting \
    --config ../config.json \
    --userdir ../ \
    --strategy MAStrategy \
    --timeframe 1h


Result for strategy MAStrategy
                                             BACKTESTING REPORT                                              
┌──────────┬────────┬──────────────┬─────────────────┬──────────────┬──────────────┬────────────────────────┐
│     Pair │ Trades │ Avg Profit % │ Tot Profit USDT │ Tot Profit % │ Avg Duration │  Win  Draw  Loss  Win% │
├──────────┼────────┼──────────────┼─────────────────┼──────────────┼──────────────┼────────────────────────┤
│ BTC/USDT │     30 │        -0.28 │          -0.265 │        -2.65 │     13:30:00 │    7     0    23  23.3 │
│    TOTAL │     30 │        -0.28 │          -0.265 │        -2.65 │     13:30:00 │    7     0    23  23.3 │
└──────────┴────────┴──────────────┴─────────────────┴──────────────┴──────────────┴────────────────────────┘
                                           LEFT OPEN TRADES REPORT                                           
┌──────────┬────────┬──────────────┬─────────────────┬──────────────┬──────────────┬─────

2026-01-06 16:39:25,320 - freqtrade - INFO - freqtrade 2026.1-dev-0b4ff3f
2026-01-06 16:39:28,544 - freqtrade.configuration.load_config - INFO - Using config: ../config.json ...
2026-01-06 16:39:28,546 - freqtrade.loggers - INFO - Enabling colorized output.
2026-01-06 16:39:28,546 - freqtrade.loggers - INFO - Logfile configured
2026-01-06 16:39:28,546 - freqtrade.loggers - INFO - Verbosity set to 0
2026-01-06 16:39:28,547 - freqtrade.configuration.configuration - INFO - Parameter -i/--timeframe detected ... Using timeframe: 1h ...
2026-01-06 16:39:28,547 - freqtrade.configuration.configuration - INFO - Using max_open_trades: 3 ...
2026-01-06 16:39:28,549 - freqtrade.configuration.configuration - INFO - Using user-data directory: .. ...
2026-01-06 16:39:28,549 - freqtrade.configuration.configuration - INFO - Using data directory: ..\data\okx ...
2026-01-06 16:39:28,550 - freqtrade.configuration.configuration - INFO - Parameter --cache=day detected ...
2026-01-06 16:39:28,551 - freqtrade